# 데이터 전처리 실습: 인코딩과 스케일링 (Encoding & Scaling)

## 목표
머신러닝 알고리즘이 데이터를 잘 이해할 수 있도록 돕는 핵심 전처리 기술을 실습합니다.
1.  **인코딩 (Encoding)**: 문자열(Category) 데이터를 숫자(Numerical)로 변환합니다.
2.  **스케일링 (Scaling)**: 서로 다른 단위(Scale)를 가진 변수들의 범위를 일정하게 맞춥니다.


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.datasets import load_iris

print("All libraries imported successfully.")

All libraries imported successfully.


## 1. 인코딩 (Encoding)

### 1-1. 레이블 인코딩 (Label Encoding)
*   카테고리형 데이터를 0, 1, 2... 와 같은 정수형으로 변환합니다.
*   **주의**: 숫자의 크기(순서)가 모델에 잘못된 영향을 줄 수 있습니다. (예: 냉장고(1) < 컴퓨터(2) ??)


In [2]:
# 예제 데이터
items = ['TV', '냉장고', '전자레인지', '컴퓨터', '선풍기', '선풍기', '믹서']

# LabelEncoder 객체 생성 및 학습
encoder = LabelEncoder()
encoder.fit(items)

# 변환 (Transform)
labels = encoder.transform(items)

print('인코딩 변환값:', labels)
print('인코딩 클래스:', encoder.classes_)

인코딩 변환값: [0 1 4 5 3 3 2]
인코딩 클래스: ['TV' '냉장고' '믹서' '선풍기' '전자레인지' '컴퓨터']


In [3]:
# 디코딩 (역변환)
print('디코딩 원본값:', encoder.inverse_transform(labels))

디코딩 원본값: ['TV' '냉장고' '전자레인지' '컴퓨터' '선풍기' '선풍기' '믹서']


### 1-2. 원-핫 인코딩 (One-Hot Encoding)
*   해당 클래스만 1, 나머지는 0으로 표현하는 희소 행렬(Sparse Matrix)을 만듭니다.
*   숫자의 크기 이슈를 해결할 수 있습니다.


In [4]:
# 2차원 데이터로 변환 (OneHotEncoder는 2D 입력을 기대함)
labels_2d = labels.reshape(-1, 1)

oh_encoder = OneHotEncoder()
oh_encoder.fit(labels_2d)
oh_labels = oh_encoder.transform(labels_2d)

print('원-핫 인코딩 데이터 (Sparse Matrix):\n', oh_labels)
print('\n원-핫 인코딩 데이터 (Array):\n', oh_labels.toarray())
print('\n데이터 차원(Shape):', oh_labels.shape)

원-핫 인코딩 데이터 (Sparse Matrix):
 <Compressed Sparse Row sparse matrix of dtype 'float64'
	with 7 stored elements and shape (7, 6)>
  Coords	Values
  (0, 0)	1.0
  (1, 1)	1.0
  (2, 4)	1.0
  (3, 5)	1.0
  (4, 3)	1.0
  (5, 3)	1.0
  (6, 2)	1.0

원-핫 인코딩 데이터 (Array):
 [[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]

데이터 차원(Shape): (7, 6)


**Pandas의 `get_dummies` 활용**: 사실 사이킷런보다 판다스가 더 간편할 때가 많습니다.

In [5]:
df_items = pd.DataFrame({'item': items})
pd.get_dummies(df_items)

,item_TV,item_냉장고,item_믹서,item_선풍기,item_전자레인지,item_컴퓨터
0,True,False,False,False,False,False
1,False,True,False,False,False,False
2,False,False,False,False,True,False
3,False,False,False,False,False,True
4,False,False,False,True,False,False
5,False,False,False,True,False,False
6,False,False,True,False,False,False


## 2. 피처 스케일링 (Feature Scaling)

데이터의 단위(Scale)가 다르면 모델 학습이 불안정해질 수 있습니다.

### 2-1. 표준화 (StandardScaler)
*   **평균을 0, 분산을 1**로 만듭니다. (정규분포화)
*   SVM, 로지스틱 회귀 등에서 필수적입니다.


In [6]:
# 붓꽃 데이터 로드
iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

print('--- 스케일링 전 평균(Mean) ---')
print(iris_df.mean())
print('\n--- 스케일링 전 분산(Var) ---')
print(iris_df.var())

--- 스케일링 전 평균(Mean) ---
sepal length (cm)    5.843333
sepal width (cm)     3.057333
petal length (cm)    3.758000
petal width (cm)     1.199333
dtype: float64

--- 스케일링 전 분산(Var) ---
sepal length (cm)    0.685694
sepal width (cm)     0.189979
petal length (cm)    3.116278
petal width (cm)     0.581006
dtype: float64


In [7]:
scaler = StandardScaler()

# fit()과 transform()을 한 번에 수행
iris_scaled = scaler.fit_transform(iris_df)

# DataFrame으로 변환하여 확인
iris_df_scaled = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)

print('--- 스케일링 후 평균(Mean) ---')
print(iris_df_scaled.mean()) # 0에 근사
print('\n--- 스케일링 후 분산(Var) ---')
print(iris_df_scaled.var())  # 1에 근사

--- 스케일링 후 평균(Mean) ---
sepal length (cm)   -1.690315e-15
sepal width (cm)    -1.842970e-15
petal length (cm)   -1.698641e-15
petal width (cm)    -1.409243e-15
dtype: float64

--- 스케일링 후 분산(Var) ---
sepal length (cm)    1.006711
sepal width (cm)     1.006711
petal length (cm)    1.006711
petal width (cm)     1.006711
dtype: float64


### 2-2. 정규화 (MinMaxScaler)
*   데이터를 **0과 1 사이**의 값으로 변환합니다.
*   음수가 있으면 -1과 1 사이로 변환될 수도 있습니다.


In [8]:
scaler = MinMaxScaler()
iris_minmax = scaler.fit_transform(iris_df)
iris_df_minmax = pd.DataFrame(data=iris_minmax, columns=iris.feature_names)

print('최솟값(Min):')
print(iris_df_minmax.min())
print('\n최댓값(Max):')
print(iris_df_minmax.max())

최솟값(Min):
sepal length (cm)    0.0
sepal width (cm)     0.0
petal length (cm)    0.0
petal width (cm)     0.0
dtype: float64

최댓값(Max):
sepal length (cm)    1.0
sepal width (cm)     1.0
petal length (cm)    1.0
petal width (cm)     1.0
dtype: float64


### 💡 학습 데이터와 테스트 데이터의 스케일링 주의점
*   반드시 **학습 데이터(Train Set)로 `fit()`** 하고, 그 기준으로 **테스트 데이터(Test Set)를 `transform()`** 해야 합니다.
*   테스트 데이터로 다시 `fit()`을 하면 안 됩니다! (데이터 유출 문제)


In [9]:
# 예시
train_array = np.arange(0, 11).reshape(-1, 1) # 0~10
test_array = np.arange(0, 6).reshape(-1, 1)  # 0~5

scaler = MinMaxScaler()

# 1. Train 데이터로 학습 (0 -> 0, 10 -> 1)
scaler.fit(train_array)
train_scaled = scaler.transform(train_array)

print('Train 원본:', train_array.flatten())
print('Train 변환:', train_scaled.flatten())

# 2. Test 데이터는 Train의 기준(0~10)을 그대로 적용해야 함
# 5는 Train 기준(0~10)의 절반이므로 0.5가 되어야 함
test_scaled = scaler.transform(test_array) 

print('\nTest 원본:', test_array.flatten())
print('Test 변환:', test_scaled.flatten()) # 0, 0.1, 0.2 ... 0.5 가 나와야 정상

Train 원본: [ 0  1  2  3  4  5  6  7  8  9 10]
Train 변환: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]

Test 원본: [0 1 2 3 4 5]
Test 변환: [0.  0.1 0.2 0.3 0.4 0.5]


## 3. 인사이트 도출 (Insights)

### 🌟 Key Takeaways
*   **인코딩의 선택**:
    *   **Label Encoding**: 간편하지만, 숫자 크기(1 < 2 < 3)가 모델에 잘못된 힌트(순서/랭킹)를 줄 수 있어 트리 계열 외에는 주의해야 합니다.
    *   **One-Hot Encoding**: 범주 간의 독립성을 보장하므로, 선형 회귀나 신경망 등 대부분의 모델에서 더 안전합니다.
*   **스케일링의 필수성**: 단위(cm, kg 등)가 다르면 숫자가 큰 변수가 모델을 지배할 수 있습니다. `StandardScaler`나 `MinMaxScaler`로 공정한 비교 환경을 만들어야 합니다.
*   **Data Leakage 방지**: 스케일링 기준(`fit`)은 반드시 **Train Data**여야 합니다. Test Data는 그 기준에 맞춰 `transform`만 해야 함을 배웠습니다.

### 🔬 Try More
*   **변환 전후 비교**: 스케일링을 하지 않은 데이터로 모델을 학습시켰을 때와, 스케일링 후의 성능 차이를 직접 실험해보세요. (특히 로지스틱 회귀나 SVM에서)
*   **RobustScaler**: 이상치(Outlier)에 강한 `RobustScaler`를 `StandardScaler` 대신 사용해보고 평균/분산 변화를 비교해보세요.
